# Term Proximity Retrieval Model in SQL using DuckDB and MonetDBLite

This notebook is a work in progress for the Information Retrieval course research project. A number of retrievel models are implemented in SQL queries that can be run with two database systems.

## Index

A preliminary index of the Robust04 data set was first [downloaded from Jimmy Lin's Dropbox](https://www.dropbox.com/s/mdoly9sjdalh44x/lucene-index.robust04.pos%2Bdocvectors%2Brawdocs.tar.gz). Then the [OldDog](https://github.com/Chriskamphuis/olddog) code from Chris Kamphuis and Arjen de Vries was modified to work with more than one leaf. Their code was further modified to store the collection frequency of each term and the term frequency of each term in each document. The modified code was run, which resulted in three CSV tables. 

The *dict* table houses termid, term, document frequency, and collection frequency data. The *docs* table houses name, docid, and document length information. And finally, the *terms* table houses the termid, docid, position, and term frequency data.

These three tables are put in an archive, so that they can be easily [downloaded from Dropbox](https://www.dropbox.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar).

In [1]:
!wget -O Robust04.rar https://dl.dropboxusercontent.com/s/kzu6yuxt2d1wzww/Robust04Tables.rar?dl=0
!unrar e -o+ Robust04.rar

--2019-12-11 11:46:59--  https://dl.dropboxusercontent.com/s/kzu6yuxt2d1wzww/Robust04Tables.rar?dl=0
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.82.6, 2620:100:6032:6::a27d:5206
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.82.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 724523928 (691M) [application/rar]
Saving to: ‘Robust04.rar’

Robust04.rar         14%[=>                  ] 102.62M  --.-KB/s    in 8m 57s  

2019-12-11 11:55:57 (196 KB/s) - Connection closed at byte 107609079. Retrying.

--2019-12-11 11:55:58--  (try: 2)  https://dl.dropboxusercontent.com/s/kzu6yuxt2d1wzww/Robust04Tables.rar?dl=0
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.82.6|:443... connected.
HTTP request sent, awaiting response... 206 PARTIAL CONTENT
Length: 724523928 (691M), 616914849 (588M) remaining [application/rar]
Saving to: ‘Robust04.rar’

Robust04.rar        100%[++================

In [0]:
import os
from datetime import datetime

## Databases

Both DuckDB and MonetDBLite will be used to test the differences between them on a number of dimensions, during the evaluation phase of the research.

Google Colab does not pre-install these packages, so that is why we need to do `pip install`. This takes about 5 minutes for DuckDB, so please already run the next cell, before reading on.

Here, we have made two classes that do the necessary tasks that we need the databases to do.

MonetDB cannot be used, as it requires an external Java MonetDB server, and the project was to be made with the Python APIs. That leaves us with MonetDBLite. For some reason, they have taken out the Python DB API for MonetDBLite (the `Cursor` class and `cursor.execute()` function). We could enable it by modifying the source code, but the program also needs to work for other people on other computers. So we just stuck to the Simple API of MonetDBLite. Another titbit, MonetDBLite cannot be initialized in the memory. If you go over some memory bandwith threshold, the whole database stops working. So, MonetDBLite needs to be initialized in storage. The performance difference is not that big on Google Colab, but it did matter on a local runtime.

When initializing the database objects, the index tables are automatically added. So you only have to initialize them once.

In [0]:
!pip install duckdb
import duckdb

class DuckDB(object):
  """
  Class that houses all the DuckDB functionalities.

  Attributes:
    c         = [Cursor] database cursor of DuckDB
    C         = [int] number of indexed terms
    N         = [int] number of indexed documents
    avgdl     = [float] average number of terms per document
    len_query = [int] number of terms in current search query
  """
  def __init__(self,
               database=':memory:',
               dict='dict.csv',
               docs='docs.csv',
               terms='terms.csv'):
    """
    Initializes DuckDB database with index and statistics.

    Args:
      database = [str] database path
      dict     = [str] filename for dictionary CSV
      docs     = [str] filename for documents CSV
      terms    = [str] filename for terms CSV
    """
    # initialize database
    con = duckdb.connect(database)
    self.c = con.cursor()

    # copy dictionary CSV into DuckDB database
    self.c.execute("CREATE TABLE dict(termid INTEGER "
                                    ",term   VARCHAR "
                                    ",df     INTEGER "
                                    ",cf     INTEGER)")
    self.c.execute("COPY dict "
                   "FROM '" + dict + "' "
                   "WITH DELIMITER '|'")
    
    # copy documents CSV into DuckDB database
    self.c.execute("CREATE TABLE docs(name  VARCHAR "
                                    ",docid INTEGER "
                                    ",len   INTEGER "
                                    ",temp  INTEGER)")
    self.c.execute("COPY docs "
                   "FROM '" + docs + "' "
                   "WITH DELIMITER '|'")
    
    # copy terms CSV into DuckDB database
    self.c.execute("CREATE TABLE terms(termid INTEGER "
                                     ",docid  INTEGER "
                                     ",pos    INTEGER "
                                     ",tf     INTEGER)")
    self.c.execute("COPY terms "
                   "FROM '" + terms + "' "
                   "WITH DELIMITER '|'")
    
    # compute standard index statistics
    self.C = self._C()
    self.N = self._N()
    self.avgdl = self._avgdl()
    self.len_query = 0
    
  def make_query(self, *args: str):
    """
    Makes query table in DuckDB database filled with query terms.
    
    Args:
      args = [[str]] concatenation of strings to be made into a query
    """
    # convert search query in to SQL query
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    # make new or replace old query table
    self.c.execute("DROP TABLE IF EXISTS query")
    self.c.execute("CREATE TABLE query(term VARCHAR)")
    self.c.execute("INSERT INTO query VALUES " + query)

    # bookkeeping
    self.len_query = len(args)

  def make_queries(self, queries, qrels):
    """
    Makes queries and qrels tables in DuckDB database filled with
    queryid, term pairs and queryid, docid relevance, respectively.

    Args:
      queries = [str] filename for search queries CSV
      qrels   = [str] filename for relevance judgements CSV
    """
    # copy queries CSV into DuckDB database
    self.c.execute("DROP TABLE IF EXISTS queries")
    self.c.execute("CREATE TABLE queries(queryid INTEGER "
                                       ",term    VARCHAR "
                                       ",len     INTEGER)")
    self.c.execute("COPY queries "
                   "FROM '" + queries + "' "
                   "WITH DELIMITER '|'")
    
    # copy relevance judgements CSV into DuckDB database
    self.c.execute("DROP TABLE IF EXISTS qrels")
    self.c.execute("CREATE TABLE qrels(queryid INTEGER "
                                     ",name    VARCHAR "
                                     ",rel     INTEGER)")
    self.c.execute("COPY qrels "
                   "FROM '" + qrels + "' "
                   "WITH DELIMITER '|'")

  def execute_query(self, query):
    """
    Executes SQL query on DuckDB database.

    Args:
      query = [str] the SQL query to be executed by DuckDB

    Returns [DataFrame]:
      The output of the execution as a Pandas DataFrame object.
    """
    out = self.c.execute(query)
    return out.fetchdf()

  def _C(self):
    """ 
    Gets total number of terms in the index.
    
    Returns [int]:
      Total number of indexed terms.
    """
    C = self.c.execute("SELECT SUM(dict.cf) "
                       "FROM dict")
    return C.fetchdf().iloc[0, 0]

  def _N(self):
    """
    Gets number of documents in the index.

    Returns [int]:
      Number of indexed documents.
    """
    N = self.c.execute("SELECT COUNT(*) "
                       "FROM docs")
    return N.fetchdf().iloc[0, 0]

  def _avgdl(self):
    """
    Gets average number of terms per document in the index.

    Returns [float]:
      Average length of indexed documents.
    """
    avgdl = self.c.execute("SELECT AVG(docs.len) "
                           "FROM docs")
    return avgdl.fetchdf().iloc[0, 0]

In [3]:
!pip install monetdblite
import monetdblite as m
import pandas as pd

class MonetDBLite(object):
  """ 
  Class that houses all the MonetDBLite functionalities. 

  Attributes:
    C     = [int] number of indexed terms
    N     = [int] number of indexed documents
    avgdl = [float] average number of terms per document
    len_query = [int] number of terms in current search query
  """
  def __init__(self,
               database='/tmp/MonetDBLite Database',
               dict='dict.csv',
               docs='docs.csv',
               terms='terms.csv'):
    """
    Initializes MonetDBLite database with index.

    Args:
      database = [str] database path
      dict     = [str] filename for dictionary CSV
      docs     = [str] filename for documents CSV
      terms    = [str] filename for terms CSV
    """
    # MonetDBLite expects an absolute path
    dict = os.path.join('/content', dict)
    docs = os.path.join('/content', docs)
    terms = os.path.join('/content', terms)

    # initialize database
    m.init(database)

    # copy dictionary CSV into MonetDBLite database
    m.sql("CREATE TABLE dict(termid INTEGER "
                           ",term   VARCHAR(99) "
                           ",df     INTEGER "
                           ",cf     INTEGER)")
    m.sql("COPY INTO dict "
          "FROM '" + dict + "' "
          "USING DELIMITERS '|'")
    
    # copy documents CSV into MonetDBLite database
    m.sql("CREATE TABLE docs(name  VARCHAR(99) "
                           ",docid INTEGER "
                           ",len   INTEGER "
                           ",temp  INTEGER)") 
    m.sql("COPY INTO docs "
          "FROM '" + docs + "' "
          "USING DELIMITERS '|'")
    
    # copy terms CSV into MonetDBLite database
    m.sql("CREATE TABLE terms(termid INTEGER "
                            ",docid  INTEGER "
                            ",pos    INTEGER "
                            ",tf     INTEGER)")
    m.sql("COPY INTO terms "
          "FROM '" + terms + "' "
          "USING DELIMITERS '|'")
    
    # compute standard index statistics
    self.C = self._C()
    self.N = self._N()
    self.avgdl = self._avgdl()
    self.len_query = 0
    
  def make_query(self, *args: str):
    """
    Makes query table in MonetDBLite database filled with query terms.
    
    Args:
      args = [[str]] concatenation of strings to be made into a query
    """
    # convert search query in to SQL query
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    # make new or replace old query table
    m.sql("DROP TABLE IF EXISTS query")
    m.sql("CREATE TABLE query(term VARCHAR(99))")
    m.sql("INSERT INTO query VALUES " + query)

    # bookkeeping
    self.len_query = len(args)

  def make_queries(self, queries, qrels):
    """
    Makes queries and qrels tables in MonetDBLite database filled with
    queryid, term pairs and queryid, docid relevance, respectively.

    Args:
      queries = [str] filename for search queries CSV
      qrels   = [str] filename for relevance judgements CSV
    """
    # MonetDBLite expects an absolute path
    queries = os.path.join('/content', queries)
    qrels = os.path.join('/content', qrels)

    # copy queries CSV into MonetDBLite database
    m.sql("DROP TABLE IF EXISTS queries")
    m.sql("CREATE TABLE queries(queryid INTEGER "
                              ",term    VARCHAR(99) "
                              ",len     INTEGER)")
    m.sql("COPY INTO queries "
          "FROM '" + queries + "' "
          "USING DELIMITERS '|'")
    
    # copy relevance judgements CSV into MonetDBLite database
    m.sql("DROP TABLE IF EXISTS qrels")
    m.sql("CREATE TABLE qrels(queryid INTEGER "
                            ",name    VARCHAR(99) "
                            ",rel     INTEGER)")
    m.sql("COPY INTO qrels "
          "FROM '" + qrels + "' "
          "USING DELIMITERS '|'")

  def execute_query(self, query):
    """
    Executes SQL query on MonetDBLite database.

    Args:
      query = [str] the SQL query to be executed by MonetDBLite

    Returns [DataFrame]:
      The output of the execution as a Pandas DataFrame object.
    """
    out = m.sql(query)
    return pd.DataFrame.from_dict(out)

  def _C(self):
    """ 
    Gets total number of terms in the index.
    
    Returns [int]:
      Total number of indexed terms.
    """
    C = m.sql("SELECT SUM(dict.cf) "
              "FROM dict")
    return pd.DataFrame.from_dict(C).iloc[0, 0]

  def _N(self):
    """
    Gets number of documents in the index.

    Returns [int]:
      Number of indexed documents.
    """
    N = m.sql("SELECT COUNT(*) "
              "FROM docs")
    return pd.DataFrame.from_dict(N).iloc[0, 0]

  def _avgdl(self):
    """
    Gets average number of terms per document in the index.

    Returns [float]:
      Average length of indexed documents.
    """
    avgdl = m.sql("SELECT AVG(docs.len) "
                  "FROM docs")
    return pd.DataFrame.from_dict(avgdl).iloc[0, 0]

     |████████████████████████████████| 7.3MB 2.3MB/s 


## Retriever

The next class houses the function for retrieving the relevant documents given a number of options, `retrieve()`. The private methods (starting with an underscore) could really use some help. So please only use the `retrieve()` function further into the file. Perhaps we will clean up the mess later.

In [0]:
class Retriever(object):
  """ Class to do document retrieval with term proximity using databases. """     
  def retrieve(self,
               query, 
               db,
               con_query=True, 
               pre_select='kld', 
               tp=True,
               k=30,
               sum=True,
               mu=2000, # totally not sure about this hyper-parameter
               k1=1.2,
               b=0.75,
               num_docs=20,
               max_span=5,
               get_time=False,
               verbose=True):
    """
    Function that retreives documents with a Retrieval Status Value (RSV)
    based on term-proximity (TP) weighting, Okapi BM25 or Kullback-Leibler
    Divergence. When opting for TP, k documents can be pre-selected with
    the Okapi BM25 or Kullback-Leibler Divergence retrieval models.

    Args:
      query      = [[str]] the tokenized and normalied query
      db         = [DuckDB|MonetDBLite] database that stores the index
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      tp         = [bool] whether to do the term proximity at all
      k          = [int] maximum number of documents to retrieve with
                         the pre-selection retrieval model
      sum        = [bool] whether to sum the pre-selection and term 
                          proximity scores for the final score
      mu         = [float] hyper-parameter for the KLD retrieval model
      k1         = [float] hyper-parameter for Okapi BM25
      b          = [float] hyper-parameter for Okapi BM25
      num_docs   = [int] maximum number of documents to retrieve
      max_span   = [int] maximum distance, in number of terms, for a term
                         pair to be included in the term proximity score
      get_time   = [bool] whether to return the elapsed time with the output
      verbose    = [bool] whether to print SQL query and elapsed time

    Returns [DataFrame]:
      The Pandas DataFrame output.
      When `get_time=True`, tuple (Pandas DataFrame, datetime object).
    """
    # add the search query as a table to the database
    db.make_query(*query)

    # determine the SQL query
    sql = (self._qterms(pre_select, tp) +
           self._qtermstf(pre_select, tp) +
           self._condocs(db, con_query, tp) +
           self._pre_select_subscores(db, con_query, pre_select, mu, k1, b) +
           self._topkdocs(pre_select, k) + 
           self._pairs(con_query, pre_select, tp, max_span) +
           self._tpscores(db, tp, k1, b) +
           self._scores(pre_select, tp, sum, num_docs))
    if verbose:
      print('Query: {}'.format(sql))
    
    # get the elapsed time and the results after executing the SQL query
    time = datetime.now()
    out = db.execute_query(sql)
    time_delta = datetime.now() - time
    if verbose:
      print('Query time: {}'.format(time_delta))

    if get_time:
      return out, time_delta
    else:
      return out

  def retrieve_all(self,
                   queries,
                   qrels,
                   db,
                   con_query=True, 
                   pre_select='kld', 
                   tp=True,
                   k=30,
                   sum=True,
                   mu=2000, # totally not sure about this hyper-parameter
                   k1=1.2,
                   b=0.75,
                   num_docs=20,
                   max_span=5):
    """
    Function that retrieves a document ranking for all queries with a
    Retrieval Status Value (RSV) based on term-proximity (TP) weighting,
    Okapi BM25 or Kullback-Leibler Divergence. When opting for TP, k
    documents can be pre-selected with the Okapi BM25 or Kullback-Leibler
    Divergence retrieval models. The relevance judgements are also added.

    Args:
      queries    = [str] filename for search queries CSV
      qrels      = [str] filename for relevance judgements CSV
      db         = [DuckDB|MonetDBLite] database that stores the index
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      tp         = [bool] whether to do the term proximity at all
      k          = [int] maximum number of documents to retrieve with
                         the pre-selection retrieval model per query
      sum        = [bool] whether to sum the pre-selection and term 
                          proximity scores for the final score
      mu         = [float] hyper-parameter for the KLD retrieval model
      k1         = [float] hyper-parameter for Okapi BM25
      b          = [float] hyper-parameter for Okapi BM25
      num_docs   = [int] maximum number of documents to retrieve per query
      max_span   = [int] maximum distance, in number of terms, for a term
                         pair to be included in the term proximity score

    Returns [DataFrame]:
      The Pandas DataFrame output.
    """
    # add the queries and qrels tables to the database
    db.make_queries(queries, qrels)

    # determine the SQL query
    sql = (self._qterms(pre_select, tp, True) +
           self._qtermstf(pre_select, tp, True) +
           self._condocs(db, con_query, tp, True) +
           self._pre_select_subscores(db, con_query, pre_select, mu, k1, b, True) +
           self._topkdocs(pre_select, k, True) + 
           self._pairs(con_query, pre_select, tp, max_span, True) +
           self._tpscores(db, tp, k1, b, True) +
           self._scores(pre_select, tp, sum, num_docs, True) +
           self._qrels())    
    print('Query: {}'.format(sql))

    # get the elapsed time and the results after executing the SQL query
    time = datetime.now()
    out = db.execute_query(sql)
    time_delta = datetime.now() - time
    print('Query time: {}'.format(time_delta))

    return out

  def nr_relevant_documents(self, queries, qrels, db):
    """Function that retrieves the number of relevant document for each
       search query.

    Args:
      queries = [str] filename for search queries CSV
      qrels   = [str] filename for relevance judgements CSV
      db      = [DuckDB|MonetDBLite] database that stores the index

    Returns [DataFrame]:
      The Pandas DataFrame output with the queryid and numreldocs columns.
    """
    # add the queries and qrels tables to the database
    db.make_queries(queries, qrels)

    # determine the SQL query
    sql = ("SELECT qrels.queryid " +
                 ",COUNT(qrels.name) AS numreldocs "
           "FROM qrels "
           "GROUP BY qrels.queryid")
    print('Query: {}'.format(sql))

    # get the results after executing the SQL query
    out = db.execute_query(sql)

    return out

  def _qterms(self, pre_select, tp, all=False):
    """ 
    Get the SQL query that will retrieve the rows in the terms file
    belonging to the query terms, including the positional information.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model;
                   Kullback-Leibler Divergence retrieval model also needs
                   collection frequency information of each term  
      tp         = [bool] whether to do the term proximity
      all        = [bool] whether to retrieve a document ranking for
                          all queries

    Returns [str]:
      SQL query as string.
    """
    query = ("WITH qtermids "
                  "AS (SELECT dict.termid "
                            ",dict.df "
                            "{}"
                      "FROM dict "
                      "{}"
                      ") "
             "{}")
    
    if tp:
      query = query.format("{}",
                           "{}",
                           ", qterms "
                                "AS (SELECT terms.termid "
                                          ",terms.docid "
                                          ",terms.pos "
                                          ",terms.tf "
                                          ",qtermids.df "
                                          "{}"
                                    "FROM terms "
                                    "JOIN qtermids "
                                    "ON terms.termid = qtermids.termid"
                                    ") ")
      if pre_select == 'kld':
        query = query.format(",dict.cf "
                            "{}",
                            "{}",
                            ",qtermids.cf "
                            "{}")
      
      if all:
        return query.format(",queries.queryid "
                            ",queries.len ",
                            "JOIN queries "
                            "ON dict.term = queries.term",
                            ",qtermids.queryid "
                            ",qtermids.len ")
      else:
        return query.format("",
                            "JOIN query "
                            "ON dict.term = query.term",
                            "")  
    else:
      if pre_select == 'kld':
        query = query.format(",dict.cf "
                            "{}",
                            "{}",
                            "{}")
      
      if all:
        return query.format(",queries.queryid "
                            ",queries.len ",
                            "JOIN queries "
                            "ON dict.term = queries.term",
                            "")
      else:
        return query.format("",
                            "JOIN query "
                            "ON dict.term = query.term",
                            "")

  def _qtermstf(self, pre_select, tp, all=False):
    """
    Get the SQL query that will retrieve the rows in the terms file
    belonging to the query terms, excluding the positional information.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model;
                   Kullback-Leibler Divergence retrieval model also needs
                   collection frequency information of each term
      tp         = [bool] whether to do the term proximity
      all        = [bool] whether to retrieve a document ranking for
                          all queries

    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'none':
      return ""
    
    query = (", qtermstfrows "
                  "AS (SELECT qterms.termid "
                            ",qterms.docid "
                            ",qterms.tf "
                            "{}"
                            ",( ROW_NUMBER() "
                               "OVER(PARTITION BY {}qterms.termid, qterms.docid "
                                    "ORDER BY qterms.pos"
                                    ")"
                              ") AS row "
                      "{}" 
                      ") "
             ", qtermstf "
                  "AS (SELECT qtermstfrows.termid "
                            ",qtermstfrows.docid "
                            ",qtermstfrows.tf "
                            ",qtermstfrows.df "
                            "{}"
                      "FROM qtermstfrows "
                      "WHERE qtermstfrows.row = 1"
                      ") ")

    if tp:
      if pre_select == 'kld':
        query = query.format(",qterms.df "
                             ",qterms.cf "
                             "{}",
                             "{}",
                             "FROM qterms",
                             ",qtermstfrows.cf "
                             "{}")
      elif pre_select == 'okapi':
        query = query.format(",qterms.df "
                             "{}",
                             "{}",
                             "FROM qterms",
                             "{}")
        
      if all:
        return query.format(",qterms.queryid "
                            ",qterms.len ",
                            "qterms.queryid ,",
                            ",qtermstfrows.queryid "
                            ",qtermstfrows.len ")
      else:
        return query.format("",
                            "",
                            "")
    else:
      if pre_select == 'kld':
        query = query.format(",qtermids.df "
                             ",qtermids.cf "
                             "{}",
                             "{}",
                             "FROM terms AS qterms "
                             "JOIN qtermids "
                             "ON qterms.termid = qtermids.termid",
                             ",qtermstfrows.cf "
                             "{}")  
      elif pre_select == 'okapi':
        query = query.format(",qtermids.df "
                             "{}",
                             "{}",
                             "FROM terms AS qterms "
                             "JOIN qtermids "
                             "ON qterms.termid = qtermids.termid",
                             "{}") 
         
      if all:
        return query.format(",qtermids.queryid "
                            ",qtermids.len ",
                            "qtermids.queryid ,",
                            ",qtermstfrows.queryid "
                            ",qtermstfrows.len ")
      else:
        return query.format("",
                            "",
                            "")

  def _condocs(self, db, con_query, tp, all=False):
    """
    Get the SQL query that will retrieve the rows in the terms file
    belonging to documents that contain all the query terms.

    Args:
      db        = [DuckDB|MonetDBLite] database that stores the index
      con_query = [bool] whether all query terms need to be in
                         the document for it to be retrieved
      tp        = [bool] whether to do the term proximity
      all       = [bool] whether to retrieve a document ranking for
                          all queries

    Returns [str]:
      SQL query as string.                         
    """
    if not con_query:
      return ""
      
    query = (", condocs "
                  "AS (SELECT qterms.docid "
                      "{}"
                      "FROM {} AS qterms "
                      "GROUP BY qterms.docid "
                      "{}"
                      "HAVING COUNT(DISTINCT qterms.termid) = {}"
                      ") ")
    if tp:
      query = query.format("{}",
                           "qterms",
                           "{}",
                           "{}")
    else:
      query = query.format("{}",
                           "qtermstf",
                           "{}",
                           "{}")
      
    if all:
      return query.format(",qterms.queryid ",
                          ",qterms.queryid ",
                          "MIN(qterms.len)")
    else:
      return query.format("",
                          "",
                          db.len_query)

  def _pre_select_subscores(self, db, con_query, pre_select, mu, k1, b, all=False):
    """
    Get the SQL query that will compute a score for each 
    query term-document pair, according to the pre-selection
    retrieval model.

    Args:
      db         = [DuckDB|MonetDBLite] database that stores the index
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved     
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      mu         = [float] hyper-parameter for the KLD retrieval model
      k1         = [float] hyper-parameter for Okapi BM25
      b          = [float] hyper-parameter for Okapi BM25
      all        = [bool] whether to retrieve a document ranking for
                          all queries
    
    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'none':
      return ""

    if pre_select == 'kld':
      query = (", kldsubscores "
                    "AS (SELECT qtermstf.docid "
                              "{}"
                              ",( LOG({:f}+tf*{:f}/cf)"              
                                  "+" 
                                 "LOG(1/({:f}+docs.len))"
                                ") AS subscore "
                        "FROM qtermstf "
                        "{}"
                        "JOIN docs "
                        "ON qtermstf.docid = docs.docid"
                        ") ")
      query = query.format("{}", mu, db.C, mu, "{}")
    elif pre_select == 'okapi':
      query = (", okapisubscores "
                    "AS (SELECT qtermstf.docid "
                              "{}"
                              ",( LOG(({:f}-df+0.5)/(df+0.5))*tf*({:f}+1)"
                                  "/"
                                 "(tf+{:f}*(1-{:f}+{:f}*docs.len/{:f}))"
                                ") AS subscore "
                        "FROM qtermstf "
                        "{}"
                        "JOIN docs "
                        "ON qtermstf.docid = docs.docid"
                        ") ")
      query = query.format("{}", db.N, k1, k1, b, b, db.avgdl, "{}")

    if con_query:
      query = query.format("{}",
                           "JOIN condocs "
                           "ON qtermstf.docid = condocs.docid "
                           "{}")      
      if all:
        return query.format(",qtermstf.queryid ",
                            "AND qtermstf.queryid = condocs.queryid ")
      else:
        return query.format("",
                            "")
    else:
      if all:
        return query.format(",qtermstf.queryid ",
                            "")
      else:
        return query.format("",
                            "")

  def _topkdocs(self, pre_select, k, all=False):
    """
    Get the SQL query that will compute the pre-selection scores,
    according to the pre-selection retrieval model, and retrieve
    the top k documents.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      k          = [int] maximum number of documents retrieved with
                         the pre-selection retrieval model
      all        = [bool] whether to retrieve a document ranking for
                          all queries

    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'none':
      return ""

    query = (", topdocs "
                  "AS (SELECT subscores.docid "
                            "{}"
                            ",SUM(subscores.subscore) AS score "
                            ",( ROW_NUMBER() "
                                "OVER({}ORDER BY SUM(subscores.subscore) DESC)"
                              ") AS rank "
                      "FROM {} AS subscores "
                      "GROUP BY subscores.docid"
                      "{}"
                      ") "
             ", topkdocs "
                  "AS (SELECT topdocs.docid "
                            "{}"
                            ",topdocs.score "
                      "FROM topdocs "
                      "WHERE topdocs.rank BETWEEN 1 AND {}"
                      ") ")
    
    if all:
      query = query.format(",subscores.queryid ",
                           "PARTITION BY subscores.queryid ",
                           "{}",
                           " ,subscores.queryid",
                           ",topdocs.queryid ",
                           "{:d}")
    else:
      query = query.format("",
                           "",
                           "{}",
                           "",
                           "",
                           "{:d}")
    
    if pre_select == 'kld':
      return query.format("kldsubscores",
                          k)
    elif pre_select == 'okapi':
      return query.format("okapisubscores",
                          k)

  def _pairs(self, con_query, pre_select, tp, max_span, all=False):
    """
    Get the SQL query that will compute the term pair instance (tpi) for
    each query term pair within a span of max_span terms.

    Args:
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved                          
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model  
      tp         = [bool] whether to do the term proximity                   
      max_span   = [int] the maximum span, in terms, of a term pair to
                        include in the term proximity score
      all        = [bool] whether to retrieve a document ranking for
                          all queries

    Returns [str]:
      SQL query as string.
    """
    if not tp:
      return ""

    query = (", pairs "
                  "AS (SELECT qterms1.termid AS termid1 "
                            ",qterms2.termid AS termid2 "
                            "{}"
                            ",qterms1.docid "
                            ",1.0/(qterms1.pos-qterms2.pos) AS tpi "
                            ",( CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                    "ELSE qterms2.df "
                               "END"
                              ") AS maxdf "
                            ",( ROW_NUMBER() "
                               "OVER(PARTITION BY qterms1.termid "
                                                ",qterms2.termid "
                                                "{}"
                                                ",qterms1.docid "
                                    "ORDER BY qterms1.pos)"
                              ") AS row "
                      "FROM qterms AS qterms1 "
                      "{}"
                      "{}"
                      "JOIN qterms AS qterms2 "
                      "ON qterms1.docid = qterms2.docid AND "
                         "{}"
                         "NOT qterms1.termid = qterms2.termid AND "
                         "qterms1.pos-qterms2.pos BETWEEN 1 AND {:d}"
                      ") ")
    
    if all:
      if con_query:
        query = query.format(",qterms1.queryid ",
                             ",qterms1.queryid ",
                             "JOIN condocs ON qterms1.queryid = condocs.queryid "
                             "AND qterms1.docid = condocs.docid ",
                             "{}",
                             "qterms1.queryid = qterms2.queryid AND ",
                             max_span)
      else:
        query = query.format(",qterms1.queryid ",
                             ",qterms1.queryid ",
                             "",
                             "{}",
                             "qterms1.queryid = qterms2.queryid AND ",
                             max_span)
        
      if pre_select == 'none':
        return query.format("")
      else:
        return query.format("JOIN topkdocs ON qterms1.queryid = topkdocs.queryid "
                            "AND qterms1.docid = topkdocs.docid ")        
    else:
      if con_query:
        query = query.format("",
                             "",
                             "JOIN condocs ON qterms1.docid = condocs.docid ",
                             "{}",
                             "",
                             max_span)
      else:
        query = query.format("",
                             "",
                             "",
                             "{}",
                             "",
                             max_span)
        
      if pre_select == 'none':
        return query.format("")
      else:
        return query.format("JOIN topkdocs ON qterms1.docid = topkdocs.docid ")
      
  def _tpscores(self, db, tp, k1, b, all=False):
    """
    Get the SQL query that will compute the term proximity score.

    Args:      
      db  = [DuckDB|MonetDBLite] database that stores the index
      tp  = [bool] whether to do the term proximity
      k1  = [float] hyper-parameter for Okapi BM25
      b   = [float] hyper-parameter for Okapi BM25
      all = [bool] whether to retrieve a document ranking for all queries

    Returns [str]:
      SQL query as string.
    """
    if not tp:
      return ""
      
    query = (", tpisums "
                  "AS (SELECT pairs.termid1 "
                            ",pairs.termid2 "
                            ",pairs.docid "
                            "{}"
                            ",SUM(pairs.tpi) AS tpisum "
                      "FROM pairs "              
                      "GROUP BY pairs.termid1 "
                              ",pairs.termid2 "
                              ",pairs.docid"
                              "{}"
                      ") "
            ", tpsubscores "
                  "AS (SELECT pairs.docid "
                            "{}"
                            ",( LOG(({:f}-maxdf+0.5)/(maxdf+0.5))*tpisum*({:f}+1)"
                                "/"
                              "(tpisum+{:f}*(1-{:f}+{:f}*docs.len/{:f}))"
                              ") AS tpsubscore "
                      "FROM pairs "
                      "JOIN tpisums "
                      "ON pairs.termid1 = tpisums.termid1 AND "
                        "pairs.termid2 = tpisums.termid2 AND "
                        "pairs.docid = tpisums.docid "
                        "{}"
                      "JOIN docs "
                      "ON pairs.docid = docs.docid "
                      "WHERE pairs.row = 1"
                      ") "
            ", tpscores "
                  "AS (SELECT tpsubscores.docid "
                            "{}"
                            ",SUM(tpsubscores.tpsubscore) AS tpscore "
                      "FROM tpsubscores "
                      "GROUP BY tpsubscores.docid"
                      "{}"
                      ") ")

    if all:
      return query.format(",pairs.queryid ",
                          " ,pairs.queryid",
                          ",pairs.queryid ",
                          db.N,
                          k1,
                          k1,
                          b,
                          b,
                          db.avgdl,
                          "AND pairs.queryid = tpisums.queryid ",
                          ",tpsubscores.queryid ",
                          " ,tpsubscores.queryid")
    else:
      return query.format("",
                          "",
                          "",
                          db.N,
                          k1,
                          k1,
                          b,
                          b,
                          db.avgdl,
                          "",
                          "",
                          "")

  def _scores(self, pre_select, tp, sum, num_docs, all=False):
    """ 
    Get the SQL query that will retrieve or compute the final document scores.

    Args:      
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model 
      tp         = [bool] whether to do the term proximity
      sum        = [bool] whether to sum the pre-select and term 
                          proximity scores for the final score
      num_docs   = [int] maximum number of documents to retrieve
      all        = [bool] whether to retrieve a document ranking
                          for all queries

    Returns [str]:
      SQL query as string.
    """
    if all:
      query = (", scores "
                    "AS ({}) "
                ", topndocs "
                    "AS (SELECT scores.queryid "
                              ",scores.docid "
                              ",scores.score "
                              ",scores.rank "
                        "FROM scores "
                        "WHERE scores.rank BETWEEN 1 and {:d}"
                        ") ")
      
      if not tp:
        return query.format("SELECT topkdocs.docid "
                                  ",topkdocs.queryid "
                                  ",topkdocs.score "
                                  ",( ROW_NUMBER() "
                                     "OVER(PARTITION BY topkdocs.queryid "
                                          "ORDER BY topkdocs.score DESC)"
                                    ") AS rank "
                            "FROM topkdocs",
                            num_docs)
      elif pre_select == 'none' or not sum:
        return query.format("SELECT tpscores.docid "
                                  ",tpscores.queryid "
                                  ",tpscores.tpscore AS score "
                                  ",( ROW_NUMBER() "
                                     "OVER(PARTITION BY tpscores.queryid "
                                          "ORDER BY tpscores.tpscore DESC)"
                                    ") AS rank "
                            "FROM tpscores",
                            num_docs)
      else:
        return query.format("SELECT topkdocs.docid "
                                  ",topkdocs.queryid "
                                  ",( topkdocs.score"
                                      "+"
                                    "COALESCE(tpscores.tpscore, 0)"
                                    ") AS score "
                                  ",( ROW_NUMBER() "
                                     "OVER(PARTITION BY topkdocs.queryid "
                                          "ORDER BY topkdocs.score"
                                                    "+"
                                                   "COALESCE(tpscores.tpscore, 0) "
                                                "DESC)"
                                    ") AS rank "
                            "FROM topkdocs "
                            "LEFT JOIN tpscores "
                            "ON topkdocs.docid = tpscores.docid AND "
                               "topkdocs.queryid = tpscores.queryid",
                            num_docs)
    else:           
      query = (", scores "
                    "AS ({}) "
                "SELECT scores.docid "
                      ",scores.score "
                      ",scores.rank "
                "FROM scores "
                "WHERE scores.rank BETWEEN 1 AND {:d}")
      
      if not tp:
        return query.format("SELECT topkdocs.docid "
                                  ",topkdocs.score "
                                  ",( ROW_NUMBER() "
                                     "OVER(ORDER BY topkdocs.score DESC)"
                                    ") AS rank "
                            "FROM topkdocs",
                            num_docs)
      elif pre_select == 'none' or not sum:
        return query.format("SELECT tpscores.docid "
                                  ",tpscores.tpscore AS score "
                                  ",( ROW_NUMBER() "
                                     "OVER(ORDER BY tpscores.tpscore DESC)"
                                    ") AS rank "
                            "FROM tpscores",
                            num_docs)
      else:
        return query.format("SELECT topkdocs.docid "
                                  ",( topkdocs.score"
                                      "+"
                                    "COALESCE(tpscores.tpscore, 0)"
                                    ") AS score "
                                  ",( ROW_NUMBER() "
                                     "OVER(ORDER BY topkdocs.score"
                                                    "+"
                                                   "COALESCE(tpscores.tpscore, 0) "
                                          "DESC)"
                                    ") AS rank "
                            "FROM topkdocs "
                            "LEFT JOIN tpscores "
                            "ON topkdocs.docid = tpscores.docid",
                            num_docs)
        
  def _qrels(self):
    """ 
    Get the SQL query that will retrieve the relevance judgement for each
    retrieved query-document pair.

    Returns [str]:
      SQL query as string.
    """
    query = ("SELECT topndocs.queryid "
                   ",docs.name "
                   ",topndocs.score "
                   ",topndocs.rank "
                   ",COALESCE(qrels.rel, 0) AS rel "
             "FROM topndocs "
             "JOIN docs "
             "ON topndocs.docid = docs.docid "
             "LEFT JOIN qrels "
             "ON topndocs.queryid = qrels.queryid AND "
                "docs.name = qrels.name")
    return query

## Initialization

Both databases are initialized, which loads the indices in the databases and makes them ready for execution. This can take about 5 minutes, so time for coffee!

In [0]:
# duck = DuckDB()
monet = MonetDBLite()

The `Retriever` class does not have a constructor, so initializing it is a bit meaningless. I still prefer to put it in a class to hide the private methods.

In [0]:
retriever = Retriever()

Download the rest of the test collection (test queries and corresponding relevance judgements).

In [7]:
!wget https://raw.githubusercontent.com/nnistelrooij/Information-Retrieval/TestQueries/data/test_queries.csv
!wget https://raw.githubusercontent.com/nnistelrooij/Information-Retrieval/TestQueries/data/qrels_binary.csv

--2019-12-11 12:25:20--  https://raw.githubusercontent.com/nnistelrooij/Information-Retrieval/TestQueries/data/test_queries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9292 (9.1K) [text/plain]
Saving to: ‘test_queries.csv’

test_queries.csv    100%[===================>]   9.07K  --.-KB/s    in 0s      

2019-12-11 12:25:21 (121 MB/s) - ‘test_queries.csv’ saved [9292/9292]

--2019-12-11 12:25:22--  https://raw.githubusercontent.com/nnistelrooij/Information-Retrieval/TestQueries/data/qrels_binary.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 2

### Running the retriever

In [0]:
# duck_rels = retriever.retrieve_all('test_queries.csv', 'qrels.csv', duck, con_query=False, num_docs=30)

In [8]:
monet_bm25 = retriever.retrieve_all('test_queries.csv', 'qrels_binary.csv', 
                                    monet, con_query=False, pre_select='okapi', 
                                    k=100, num_docs=100, tp=False)
monet_kld = retriever.retrieve_all('test_queries.csv', 'qrels_binary.csv', 
                                    monet, con_query=False, pre_select='kld', 
                                    k=100, num_docs=100, tp=False)
monet_tp = retriever.retrieve_all('test_queries.csv', 'qrels_binary.csv', 
                                    monet, con_query=False, pre_select='none', 
                                    k=100, num_docs=100, tp=True)

Query: WITH qtermids AS (SELECT dict.termid ,dict.df ,queries.queryid ,queries.len FROM dict JOIN queries ON dict.term = queries.term) , qtermstfrows AS (SELECT qterms.termid ,qterms.docid ,qterms.tf ,qtermids.df ,qtermids.queryid ,qtermids.len ,( ROW_NUMBER() OVER(PARTITION BY qtermids.queryid ,qterms.termid, qterms.docid ORDER BY qterms.pos)) AS row FROM terms AS qterms JOIN qtermids ON qterms.termid = qtermids.termid) , qtermstf AS (SELECT qtermstfrows.termid ,qtermstfrows.docid ,qtermstfrows.tf ,qtermstfrows.df ,qtermstfrows.queryid ,qtermstfrows.len FROM qtermstfrows WHERE qtermstfrows.row = 1) , okapisubscores AS (SELECT qtermstf.docid ,qtermstf.queryid ,( LOG((528030.000000-df+0.5)/(df+0.5))*tf*(1.200000+1)/(tf+1.200000*(1-0.750000+0.750000*docs.len/330.551052))) AS subscore FROM qtermstf JOIN docs ON qtermstf.docid = docs.docid) , topdocs AS (SELECT subscores.docid ,subscores.queryid ,SUM(subscores.subscore) AS score ,( ROW_NUMBER() OVER(PARTITION BY subscores.queryid ORDER BY 

In [9]:
monet_bm25_tp = retriever.retrieve_all('test_queries.csv', 'qrels_binary.csv', 
                                    monet, con_query=False, pre_select='okapi', 
                                    k=100, num_docs=100, tp=True)
monet_kld_tp = retriever.retrieve_all('test_queries.csv', 'qrels_binary.csv', 
                                    monet, con_query=False, pre_select='kld', 
                                    k=100, num_docs=100, tp=True)

Query: WITH qtermids AS (SELECT dict.termid ,dict.df ,queries.queryid ,queries.len FROM dict JOIN queries ON dict.term = queries.term) , qterms AS (SELECT terms.termid ,terms.docid ,terms.pos ,terms.tf ,qtermids.df ,qtermids.queryid ,qtermids.len FROM terms JOIN qtermids ON terms.termid = qtermids.termid) , qtermstfrows AS (SELECT qterms.termid ,qterms.docid ,qterms.tf ,qterms.df ,qterms.queryid ,qterms.len ,( ROW_NUMBER() OVER(PARTITION BY qterms.queryid ,qterms.termid, qterms.docid ORDER BY qterms.pos)) AS row FROM qterms) , qtermstf AS (SELECT qtermstfrows.termid ,qtermstfrows.docid ,qtermstfrows.tf ,qtermstfrows.df ,qtermstfrows.queryid ,qtermstfrows.len FROM qtermstfrows WHERE qtermstfrows.row = 1) , okapisubscores AS (SELECT qtermstf.docid ,qtermstf.queryid ,( LOG((528030.000000-df+0.5)/(df+0.5))*tf*(1.200000+1)/(tf+1.200000*(1-0.750000+0.750000*docs.len/330.551052))) AS subscore FROM qtermstf JOIN docs ON qtermstf.docid = docs.docid) , topdocs AS (SELECT subscores.docid ,subscor

In [0]:
# duck_rels

In [0]:
# monet_rels

## Evaluation framework

We can download the custom implementation of the evaluation metrics from our Github repository and import the `metrics.py` Python file.

In [11]:
# update download link to master branch when it is up to date
!wget https://raw.githubusercontent.com/nnistelrooij/Information-Retrieval/TestQueries/metrics.py

import metrics
import numpy as np

--2019-12-11 12:31:25--  https://raw.githubusercontent.com/nnistelrooij/Information-Retrieval/TestQueries/metrics.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4626 (4.5K) [text/plain]
Saving to: ‘metrics.py’

metrics.py          100%[===================>]   4.52K  --.-KB/s    in 0s      

2019-12-11 12:31:25 (85.1 MB/s) - ‘metrics.py’ saved [4626/4626]



In [0]:
# from importlib import reload
# reload(metrics)

<module 'metrics' from '/content/metrics.py'>

### MAP and gMAP

The performance of the retrieval models on the test collection can be evaluated with the Mean Average Precision (MAP). Another variant of this metric is the Geometric Mean Average Precision (gMAP), which places more emphasis on low performing queries.

In [0]:
def evaluate_test_collection(df, df_rel, metric_func, **kwargs):
  """
  Evaluates the ranked lists from a test collection 
  of a retrieval model using the specified `metric_func`.

  Args:
    df          = [DataFrame] pandas DataFrame that houses
                              the output of a retrieval model
                              (required columns: `queryid`, `rank`, `rel`)
    metric_func = [function] a function that computes a score
                             for a test collection; the function
                             accepts one argument, which is a
                             nested collection of relevance judgements,
                             e.g., [[1,1,0], [0,1,1], [1,0,0], ...], or 
                             a tuple of a nested collection of relevance 
                             judgements with the number of relevant documents, 
                             and optionally a named argument k for depth
    **kwargs    = [any] extra arguments for metric_func

  Returns [int]:
    Score from `metric_func`
  """
  k = kwargs.get('k', None)
  query_ids = pd.unique(df.queryid)

  rels = []
  for query_id in query_ids:
    rel = df[df.queryid == query_id].sort_values(by=['rank']).rel.to_numpy()
    num_rel = df_rel[df_rel.queryid == query_id].numreldocs.to_numpy() # number of relevant documents
    if k:
      rels.append(rel)
    else:
      rels.append( (rel, num_rel) ) 

  score = 0
  if k:
    score = metric_func(rels, k)
  else:
    score = metric_func(rels)
  return score

### Model runs MAP and P@k evaluation


MAP needs an argument for the number of relevant documents in the collection. The following cell retrieves the number of relevant documents per query in the test collection.

In [13]:
monet_num_rel_docs = retriever.nr_relevant_documents('test_queries.csv', 'qrels_binary.csv', monet)

Query: SELECT qrels.queryid ,COUNT(qrels.name) AS numreldocs FROM qrels GROUP BY qrels.queryid


In [14]:
print(monet_num_rel_docs.queryid.to_numpy().size) # one missing query

249


In [15]:
# find missing queries
df_queries = pd.read_csv('test_queries.csv', sep='|', header=None, names=['queryid', 'qterm', 'qlen'])
queries_with_rels = pd.unique(monet_num_rel_docs.queryid)
df_queries_without_rels = df_queries[~df_queries.queryid.isin(queries_with_rels)]
queries_without_rels = pd.unique(df_queries_without_rels.queryid)
print(queries_without_rels)

[672]


The actual evaluation of the model runs follows hereafter. We remove the queries without relevance judgements, and also add queries that do have relevance judgements but no retrieved documents (i.e., Average Precision of those queries with no results should be 0.0). The latter is achieved by adding one row corresponding to that query and set the relevance to 0, which will yield an Average Precision of 0.0 for that query.

In [0]:
model_runs = [monet_bm25, monet_kld, monet_tp, monet_bm25_tp, monet_kld_tp]
model_names = ['bm25', 'kld', 'tp', 'bm25+tp', 'kld+tp']

model_runs_clean = []
for run in model_runs:
  # remove queries without relevance judgements
  run = run[~run.queryid.isin(queries_without_rels)]
  # add queries with relevance judgements but without retrieved documents
  query_ids = pd.unique(run.queryid)
  missing_queries = pd.unique(monet_num_rel_docs[~monet_num_rel_docs.queryid.isin(query_ids)].queryid)
  missing_queries_list = []
  for query in missing_queries:
      row = {}
      row.update({'queryid': query})
      row.update({'name': ''})
      row.update({'score': 0})
      row.update({'rank': 0})
      row.update({'rel': 0})
      missing_queries_list.append(row)
  df_missing_queries = pd.DataFrame(missing_queries_list) 
  run = run.append(df_missing_queries) 
  model_runs_clean.append(run)

**MAP**

In [17]:
for run, name in zip(model_runs_clean, model_names):
  nr_of_queries = pd.unique(run.queryid).size  
  MAP = evaluate_test_collection(run, monet_num_rel_docs, metrics.mean_average_precision)
  print(name + ':', MAP, f'({nr_of_queries} queries)')

bm25: 0.14649987100511055 (249 queries)
kld: 0.14861494611205617 (249 queries)
tp: 0.08612502460193515 (249 queries)
bm25+tp: 0.14694326415675082 (249 queries)
kld+tp: 0.1499366919544142 (249 queries)


**P@k**

In [18]:
k = 10
for run, name in zip(model_runs_clean, model_names):
  nr_of_queries = pd.unique(run.queryid).size
  precision_at_10 = evaluate_test_collection(run, monet_num_rel_docs, metrics.mean_precision_at_k, k=k)
  print(name + ':', precision_at_10, f'({nr_of_queries} queries)')

bm25: 0.3060240963855422 (249 queries)
kld: 0.3120481927710843 (249 queries)
tp: 0.19277108433734938 (249 queries)
bm25+tp: 0.30080321285140565 (249 queries)
kld+tp: 0.30803212851405626 (249 queries)


### Timing

In [0]:
from timeit import Timer

In [0]:
query = ['new', 'york', 'city'] # retrieved number of documents should be equal for the query

In [0]:
def run_time(query, db, k, pre_select, tp, nr_of_runs=20):
  """
  """
  assert nr_of_runs > 10, "Number of runs should be more than 10"
  times = []
  for i in np.arange(nr_of_runs):
    _, time = retriever.retrieve(query, db, 
                                 con_query=False, pre_select=pre_select, 
                                 k=k, num_docs=k, tp=tp, 
                                 get_time=True, verbose=False)
    times.append(time)
  print("Min time:", min(times))
  print("Max time:", max(times))
  print("Mean time:", np.mean(times[(nr_of_runs-10):]))

In [119]:
print("--- Okapi BM25 ---")
run_time(query=query, db=monet, k=1000, pre_select='okapi', tp=False)

--- Okapi BM25 ---
Min time: 0:00:00.682733
Max time: 0:00:00.760328
Mean time: 0:00:00.711076


In [120]:
print("--- KLD ---")
run_time(query=query, db=monet, k=1000, pre_select='kld', tp=False)

--- KLD ---
Min time: 0:00:00.594419
Max time: 0:00:00.667990
Mean time: 0:00:00.627781


In [121]:
print("--- TP ---")
run_time(query=query, db=monet, k=1000, pre_select='none', tp=True)

--- TP ---
Min time: 0:00:01.312953
Max time: 0:00:01.508580
Mean time: 0:00:01.359317


In [124]:
print("--- Okapi BM25 + TP ---")
time_bm25_tp = run_time(query=query, db=monet, k=1000, pre_select='okapi', tp=True)

--- Okapi BM25 + TP ---
Min time: 0:00:00.780822
Max time: 0:00:00.825596
Mean time: 0:00:00.806851


In [125]:
print("--- KLD + TP ---")
time_kld_tp = run_time(query=query, db=monet, k=1000, pre_select='kld', tp=True)

--- KLD + TP ---
Min time: 0:00:00.753947
Max time: 0:00:00.827427
Mean time: 0:00:00.803147


### Kendall's $\tau$

We can also quantify to what extent ranked lists differ from each other after term proximity using Kendall's $\tau$. The ranked lists must contain the same elements, i.e, output from the retrieval models should be from term proximity on top *k* documents or just the top *k* documents without term proximity. In the options of the `Retriever` functions we must set `num_docs` equal to `k` to ensure that the returned ranked lists contain the same documents, which is a requirement to compute Kendall's $\tau$.

In [0]:
def compare_ranked_lists(df_A, df_B, query_id):
  """
  Compute Kendall's tau for a specific query from the test collection.
  Inputs should be from the output of `Retriever.retrieve_all()`.

  Args:
    df_A = [DataFrame] pandas DataFrame that houses
                       the output of a retrieval model
                       (required columns: `queryid`, `rank`)
    df_B = [DataFrame] pandas DataFrame that houses
                       the output of a retrieval model
                       (required columns: `queryid`, `rank`)

  Returns [tuple]:
    Tuple of Kendall's tau and p-value
  """
  ranked_doc_names_A = df_A[df_A.queryid == query_id].sort_values(by=['rank']).name.to_numpy()
  ranked_doc_names_B = df_B[df_B.queryid == query_id].sort_values(by=['rank']).name.to_numpy()
  return metrics.kendall_tau(ranked_doc_names_A, ranked_doc_names_B)

def compare_all_ranked_lists(df_A, df_B):
  """
  Compute Kendall's tau for all queries in a test collection.
  Inputs should be from the output of `Retriever.retrieve_all()`.

  Args:
    df_A = [DataFrame] pandas DataFrame that houses
                       the output of a retrieval model
                       (required columns: `queryid`, `rank`)
    df_B = [DataFrame] pandas DataFrame that houses
                       the output of a retrieval model
                       (required columns: `queryid`, `rank`)

    Returns [[tuple]]:
      List of tuples in the form (queryid, KendalltauResult)
  """
  query_ids = pd.unique(df_A.queryid)
  results = []
  for query_id in query_ids:
    result = compare_ranked_lists(df_A, df_B, query_id)
    results.append((query_id, result))
  return results

### Kendall's $\tau$ test collection evaluation
We can compute the mean Kendall's $\tau$ for the test collection, but only for BM25 vs BM25+TP, and KLD vs KLD+TP.

In [133]:
kendall_bm25 = compare_all_ranked_lists(monet_bm25, monet_bm25_tp)
taus_bm25 = [res[0] for _, res in kendall_bm25]
mean_tau_bm25 = np.mean(taus_bm25)
print("--- BM25 ---")
print("Mean Kendall's tau:", mean_tau_bm25)

--- BM25 ---
Mean Kendall's tau: 0.7895357214396996


In [135]:
kendall_kld = compare_all_ranked_lists(monet_kld, monet_kld_tp)
taus_kld = [res[0] for _, res in kendall_kld]
mean_tau_kld = np.mean(taus_kld)
print("--- KLD ---")
print("Mean Kendall's tau:", mean_tau_kld)

--- KLD ---
Mean Kendall's tau: 0.7520225945713801


### Examples
Some examples might be obsolete, i.e., some variables that are used do not exist.

In [0]:
# We can either use MAP or gMAP here to evaluate the test collection.
# gMAP will be zero whenever at least one query has an average precision of 0.0, 
# i.e., no relevant documents retrieved.

# duck_MAP = evaluate_test_collection(duck_rels, metrics.mean_average_precision)
monet_MAP = evaluate_test_collection(monet_rels, metrics.mean_average_precision)
# print("Duck MAP:", duck_MAP)
print("monet MAP:", monet_MAP)

# duck_gMAP = evaluate_test_collection(duck_rels, metrics.geometric_mean_average_precision)
monet_gMAP = evaluate_test_collection(monet_rels, metrics.geometric_mean_average_precision)
# print("Duck gMAP:", duck_gMAP)
print("monet gMAP:", monet_gMAP)

monet MAP: 0.17466186530268535
monet gMAP: 0.0


We already have a run of monet with term proximity `monet_rels`, and now we can also get a run without term proximity.

In [0]:
monet_rels_no_tp = retriever.retrieve_all('test_queries.csv', 'qrels_binary.csv', monet, con_query=False, tp=False, num_docs=30)

Query: WITH qtermids AS (SELECT dict.termid ,dict.df ,dict.cf ,queries.queryid ,queries.len FROM dict JOIN queries ON dict.term = queries.term) , qtermstfrows AS (SELECT qterms.termid ,qterms.docid ,qterms.tf ,qtermids.df ,qtermids.cf ,qtermids.queryid ,qtermids.len ,( ROW_NUMBER() OVER(PARTITION BY qtermids.queryid ,qterms.termid, qterms.docid ORDER BY qterms.pos)) AS row FROM terms AS qterms JOIN qtermids ON qterms.termid = qtermids.termid) , qtermstf AS (SELECT qtermstfrows.termid ,qtermstfrows.docid ,qtermstfrows.tf ,qtermstfrows.df ,qtermstfrows.cf ,qtermstfrows.queryid ,qtermstfrows.len FROM qtermstfrows WHERE qtermstfrows.row = 1) , kldsubscores AS (SELECT qtermstf.docid ,qtermstf.queryid ,( LOG(0.800000+tf*174540872.000000/cf)+LOG(1/(0.800000+docs.len))) AS subscore FROM qtermstf JOIN docs ON qtermstf.docid = docs.docid) , topdocs AS (SELECT subscores.docid ,subscores.queryid ,SUM(subscores.subscore) AS score ,( ROW_NUMBER() OVER(PARTITION BY subscores.queryid ORDER BY SUM(subs

The Kendall's $\tau$ can be computed for all queries in the test collection at once. We can also take the mean of it, but that probably is not very informative.

In [0]:
results = compare_all_ranked_lists(monet_rels_no_tp, monet_rels)
print(results)

taus = [res[0] for _, res in results]
mean_tau = np.mean(taus)
print("Mean Kendall's tau:", mean_tau)

[(301, KendalltauResult(correlation=0.46206896551724136, pvalue=0.00022632769009664152)), (302, KendalltauResult(correlation=0.9954022988505749, pvalue=2.2619925772895436e-31)), (303, KendalltauResult(correlation=0.48045977011494256, pvalue=0.000118141400796487)), (306, KendalltauResult(correlation=0.9908045977011495, pvalue=3.4985485195411604e-30)), (307, KendalltauResult(correlation=0.9264367816091954, pvalue=4.0342030281130966e-21)), (308, KendalltauResult(correlation=0.9908045977011495, pvalue=3.4985485195411604e-30)), (310, KendalltauResult(correlation=0.5678160919540229, pvalue=3.2710732393140137e-06)), (311, KendalltauResult(correlation=0.6735632183908047, pvalue=1.0391221257473754e-08)), (312, KendalltauResult(correlation=0.9999999999999999, pvalue=0.002777777777777778)), (313, KendalltauResult(correlation=0.7333333333333334, pvalue=1.5287984352149481e-10)), (316, KendalltauResult(correlation=1.0, pvalue=7.539975257631811e-33)), (317, KendalltauResult(correlation=1.0, pvalue=7.

We can also do ad hoc retrieval for a single query using `retriever.retrieve()` and compute the Kendall's tau for that query over two ranked lists. Again make sure to set `num_docs` equal to `k`.

In [0]:
def compare_ranked_lists_ad_hoc(df_A, df_B):
  """
  Compute Kendall's tau for two ranked lists.
  Inputs should be from the output of `Retriever.retrieve()`.

  Args:
    df_A = [DataFrame] pandas DataFrame that houses
                       the output of a retrieval model
                       (required columns: `docid`, `rank`)
    df_B = [DataFrame] pandas DataFrame that houses
                       the output of a retrieval model
                       (required columns: `docid`, `rank`)

  Returns [tuple]:
    Tuple of Kendall's tau and p-value
  """
  ranked_doc_names_A = df_A.sort_values(by=['rank']).docid.to_numpy()
  ranked_doc_names_B = df_B.sort_values(by=['rank']).docid.to_numpy()
  return metrics.kendall_tau(ranked_doc_names_A, ranked_doc_names_B)

In [0]:
query = ['new', 'york']
monet_scores = retriever.retrieve(query, monet, pre_select='okapi', tp=False, num_docs=30)
monet_scores_tp = retriever.retrieve(query, monet, pre_select='okapi', tp=True, num_docs=30)

Query: WITH qtermids AS (SELECT dict.termid ,dict.df FROM dict JOIN query ON dict.term = query.term) , qtermstfrows AS (SELECT qterms.termid ,qterms.docid ,qterms.tf ,qtermids.df ,( ROW_NUMBER() OVER(PARTITION BY qterms.termid, qterms.docid ORDER BY qterms.pos)) AS row FROM terms AS qterms JOIN qtermids ON qterms.termid = qtermids.termid) , qtermstf AS (SELECT qtermstfrows.termid ,qtermstfrows.docid ,qtermstfrows.tf ,qtermstfrows.df FROM qtermstfrows WHERE qtermstfrows.row = 1) , condocs AS (SELECT qterms.docid FROM qtermstf AS qterms GROUP BY qterms.docid HAVING COUNT(DISTINCT qterms.termid) = 2) , okapisubscores AS (SELECT qtermstf.docid ,( LOG((528030.000000-df+0.5)/(df+0.5))*tf*(1.200000+1)/(tf+1.200000*(1-0.750000+0.750000*docs.len/330.551052))) AS subscore FROM qtermstf JOIN condocs ON qtermstf.docid = condocs.docid JOIN docs ON qtermstf.docid = docs.docid) , topdocs AS (SELECT subscores.docid ,SUM(subscores.subscore) AS score ,( ROW_NUMBER() OVER(ORDER BY SUM(subscores.subscore)

In [0]:
monet_ny_tau, monet_ny_pval = compare_ranked_lists_ad_hoc(monet_scores, monet_scores_tp)
print("Tau:", monet_ny_tau, "p-value:", monet_ny_pval)

Tau: 0.37471264367816093 p-value: 0.0032693506274705953


## Options

Now we will explain how to use this complicated `retrieve()` function by setting a number of specific options on or off.

### All options

Let's start with the default options. The query will be (`query=`)*new york* and it will be executed for both databases (`db=DuckDB` and `db=MonetDBLite`).

First, the top (`k=`)30 documents are retrieved using the Kullback-Leibler Divergence (`pre_select='kld'`) retrieval model with conjunctive queries (`con_query=True`) and a $\mu$ of 0.8 (`mu=0.8`). Then, these 30 documents are scored again with term proximity weighting (`tp=True`) based on a modified version of the [Rasolofo algorithm](https://www.researchgate.net/publication/225174089_Term_Proximity_Scoring_for_Keyword-Based_Retrieval_Systems), with a $k1$ of 1.2 (`k1=1.2`), a $b$ of 0.75 (`b=0.75`), and a maximum distance of 5 (`max_span=5`) between query terms. The scores obtained from KLD and Rasolofo are summed (`sum=True`) to arrive at the final score and ranking. Of this final ranking, 20 documents are retrieved (`num_docs=20`).

In [0]:
query = ['new', 'york']

print("DuckDB with all options")
duck_scores = retriever.retrieve(query, duck)

print("\nMonetDBLite with all options")
monet_scores = retriever.retrieve(query, monet)

DuckDB was very slow compared to MonetDBLite (approximately a factor of 35). The query that both databases execute is identical, so the time difference is purely DuckDB's shortcoming. Furthermore, the actual scores from DuckDB are different than from MonetDBLite. However, the ranking, surprisingly, is identical between the two.

In [0]:
duck_scores

In [0]:
monet_scores

### No term proximity weighting

If you only want to rank the documents based on Okapi BM25, then run the following code. All the other options, where applicable, are still the same.

In [0]:
print("DuckDB with only Okapi BM25")
duck_scores = retriever.retrieve(query, duck, pre_select='okapi', tp=False)

print("\nMonetDBLite with  only Okapi BM25")
monet_scores = retriever.retrieve(query, monet, pre_select='okapi', tp=False)

The difference in time between DuckDB and MonetDBLite is now much smaller (a factor of about 10), but the scores are still different, whereas the ranking is again identical.

In [0]:
duck_scores

In [0]:
monet_scores

### Only term proximity weighting

If you do not want to pre-select `k` documents, before computing the Rasolofo score, then run the following code.

In [0]:
print("DuckDB with only Rasolofo")
duck_scores = retriever.retrieve(query, duck, pre_select='none')

print("\nMonetDBLite with only Rasolofo")
monet_scores = retriever.retrieve(query, monet, pre_select='none')

The factor in the time difference is approximately 10. The execution time for MonetDBLite is more than previously. This makes sense; the algorithm needs to find the close query term pairs in all the documents instead of only the top 30 documents. This operation has a super-linear running time, so it takes MonetDBLite longer to produce the output.

The scores are again different, but now the ranking is also different. So DuckDB does not work for the given query.

In [0]:
duck_scores

In [0]:
monet_scores

### Bigger term pair radius

If the Rasolofo algorithm retrieves too few documents with the default parameters, the span of a term pair can be expanded to include more documents. To do that, run the below code with the query *wizard hat*.

It is slightly slower than the default span with MonetDBLite.

In [0]:
query = ['wizard', 'hat']

print("DuckDB with default term pair span")
duck_scores = retriever.retrieve(query, duck, pre_select='none')

print("\nMonetDBLite with default term pair span")
monet_scores = retriever.retrieve(query, monet, pre_select='none')

In [0]:
duck_scores

In [0]:
monet_scores

In [0]:
print("DuckDB with bigger term pair span")
duck_scores = retriever.retrieve(query, duck, pre_select='none', max_span=20)

print("\nMonetDBLite with bigger term pair span")
monet_scores = retriever.retrieve(query, monet, pre_select='none', max_span=20)

In [0]:
duck_scores

In [0]:
monet_scores

### Only TP scores

If you want that the final score is only the Rasolofo score, then set `sum=False`. So not the sum of the pre-selection score and the Rasolofo score as final score, but only the Rasolofo score. If the Rasolofo algorithm could not score a document that was retrieved by the pre-selection retrieval model, then that document will not be included in the ranking. So do be aware of that when setting this option!

In [0]:
print("DuckDB with summed scores")
duck_scores = retriever.retrieve(query, duck)

print("\nMonetDBLite with summed scores")
monet_scores = retriever.retrieve(query, monet)

In [0]:
duck_scores

In [0]:
monet_scores

In [0]:
print("DuckDB with Rasolofo scores")
duck_scores = retriever.retrieve(query, duck, sum=False)

print("\nMonetDBLite with Rasolofo scores")
monet_scores = retriever.retrieve(query, monet, sum=False)

In [0]:
duck_scores

In [0]:
monet_scores

## Conclusion

As seen by the many options, there are endless opportunities for research and evaluation. And we haven't even talked about combining options, disjunctive queries, or hyper-parameters, yet.

Our question right now is basically: why is DuckDB so slow and why does DuckDB give incorrect and unintuitive results. Analyzing the runtime of MonetDBLite was much more straightforward than for DuckDB. Is there still some hope for DuckDB, maybe in the future? Or should we only focus on MonetDBLite and maybe implement a second term proximity retrieval model, instead of comparing the two databases?